<a href="https://colab.research.google.com/github/KadinRelefourd/neuralnetworkclassifier/blob/main/classifyFINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

colab inviornment

Imports

In [35]:
import os
import cv2  # OpenCV for image processing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

Prepare Training and Testing data

In [36]:
transform = transforms.ToTensor()

train_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=True,
    transform=transform,
    download=True
)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=False,
    transform=transform,
    download=True
)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Build model

In [37]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)  # Input size: CIFAR-10 images are 32x32x3
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64, 10)  # Output size: 10 classes (CIFAR-10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input image
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.dropout(x, p=0.3)  # Apply dropout for regularization
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.dropout(x, p=0.3)
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.dropout(x, p = 0.3)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.fc5(x)
        return x

model = Net()
loss_func = nn.CrossEntropyLoss()  # CrossEntropyLoss includes softmax internally
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

Testing Function

In [38]:
def test():
    model.eval()  # Switch the model to evaluation mode (disables dropout, batch norm updates)
    correct = 0  # Initialize a counter for correctly predicted samples
    total = 0  # Initialize a counter for the total number of samples
    total_loss = 0.0  # Initialize a variable to accumulate the total loss

    with torch.no_grad():  # Disable gradient computation (saves memory and speeds up inference)
        for inputs, targets in test_loader:  # Iterate through the test dataset
            outputs = model(inputs)  # Forward pass: get predictions from the model
            loss = loss_func(outputs, targets)  # Compute the loss for the batch
            total_loss += loss.item()  # Accumulate the total loss

            _, predicted = torch.max(outputs.data, 1)  # Get the predicted class (highest probability)
            total += targets.size(0)  # Update the total number of samples processed
            correct += (predicted == targets).sum().item()  # Count correctly predicted samples

    accuracy = 100 * correct / total  # Compute the accuracy percentage
    return total_loss / len(test_loader), accuracy  # Return the average loss and accuracy


Training Accuracy

In [39]:
def train_accuracy():
    model.eval()  # Switch to evaluation mode for accuracy calculation on training data
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return 100 * correct / total

Save Model function

In [40]:
def save_model(epoch):
    if not os.path.exists('./model'):
        os.makedirs('./model')
    torch.save(model.state_dict(), './model/model.ckpt')  # Save with fixed filename
    print(f"Model saved in file: ./model/model.ckpt")

Load Model Funciont

In [41]:
def load_model(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Model loaded successfully.")

Train Function


In [42]:
def train():
    print(f"{'Loop':<8}{'Train Loss':<15}{'Train Acc %':<15}{'Test Loss':<15}{'Test Acc %':<15}")

    last_train_acc = 0  # Track the last training accuracy

    for epoch in range(10):  # Train for 10 epochs
        model.train()  # Set the model to training mode
        total_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_func(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Evaluate after each epoch
        train_acc = train_accuracy()
        test_loss, test_accuracy = test()

        # Ensure training accuracy only goes up
        if train_acc < last_train_acc:
         last_train_acc = train_acc  # Update last training accuracy

        #this
        print(f"{epoch + 1}/10   {total_loss / len(train_loader):<15.4f}{train_acc:<15.2f}{test_loss:<15.4f}{test_accuracy:<15.2f}")

        # Save the model only after the last epoch (10th epoch)
        if epoch == 9:
            save_model(epoch)


Test Function

In [43]:
def classify(image_path):
    load_model('./model/model.ckpt')  # Load the best model from training
    model.eval()

    # Read the image using OpenCV and preprocess it
    img = cv2.imread(image_path)  # Load image as a NumPy array (BGR format)

    if img is None:
        print("Error: Image not found or invalid path.")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB format (CIFAR-10 uses RGB)
    img_resized = cv2.resize(img, (32, 32))  # Resize to CIFAR-10 dimensions (32x32)

    img_tensor = transform(img_resized).unsqueeze(0)  # Convert to tensor and add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_class = torch.max(output.data, 1)

    classes = train_data.classes
    print(f'Prediction result: {classes[predicted_class.item()]}')

Command Line prompts

In [44]:
if __name__ == "__main__":
    import sys

    command = sys.argv[1]

    if command == "train":
        train()

    elif command == "test":
        image_path = sys.argv[2]
        classify(image_path)

In [45]:
import matplotlib.pyplot as plt
import os

# Save a test image from CIFAR-10's test dataset
img, label = test_data[0]  # First test image
classes = test_data.classes  # Class names

# Convert tensor to NumPy array (HWC format)
img_np = img.permute(1, 2, 0).numpy()

# Create a folder to store the image
if not os.path.exists('./test_images'):
    os.makedirs('./test_images')

# Save the image
plt.imsave(f'./test_images/test_img_0_{classes[label]}.png', img_np)
print(f"Saved: ./test_images/test_img_0_{classes[label]}.png")


Saved: ./test_images/test_img_0_cat.png


In [49]:
!python classify.py test ./test_images/test_img_0_cat.png

Files already downloaded and verified
Files already downloaded and verified
/content/classify.py:125: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(tor

Run Training

In [48]:
!python classify.py train

Files already downloaded and verified
Files already downloaded and verified
Loop    Train Loss     Train Acc %    Test Loss      Test Acc %     
1/10   1.9490         37.50          1.7693         37.18          
2/10   1.7101         40.98          1.6718         40.34          
3/10   1.6259         44.14          1.5919         42.93          
4/10   1.5696         44.87          1.5788         43.37          
5/10   1.5227         46.82          1.5428         44.41          
6/10   1.4910         47.16          1.5492         44.42          
7/10   1.4598         48.08          1.5271         45.63          
8/10   1.4347         49.91          1.4901         47.01          
9/10   1.4086         48.92          1.5360         45.12          
10/10   1.3924         52.08          1.4617         47.79          
Model saved in file: ./model/model.ckpt


train